In [1]:
import warnings
warnings.filterwarnings('ignore')

##  Config and Data

In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel, BertConfig,BertForSequenceClassification, AdamW
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import os

class Config:
    def __init__(self):
        self.num_labels = 28
        self.epochs = 10
        self.lr = 2e-5
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = 'bert-base-uncased'
        self.result_path = 'result'
        self.checkpoint_path = 'saved_dict'
        self.batch_size = 32
        self.max_length = 32
        self.score_key = 'f1_score'

config = Config()
# 定义Tokenizer
tokenizer = BertTokenizer.from_pretrained(config.model)

# 自定义数据集类，用于PyTorch的DataLoader
class TextDataset(Dataset):
    def __init__(self, tokenizer, max_length, num_class=28, mode='train'):
        self.data = pd.read_csv("data/{}.tsv".format(mode), sep='\t', header=None)
        self.data.columns = ['text', 'labels', 'id']
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.num_class = num_class

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data['text'][idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        labels = self.data['labels'][idx].split(',')
        return { 
            'input_ids': encoding['input_ids'].squeeze(0), 
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(convert_onehot(labels), dtype=torch.float32)
        }
    
def convert_onehot(labels):
    res = [0] * config.num_labels
    for label in labels:
        res[int(label)] = 1
    return res

train_dataset = TextDataset(tokenizer, max_length=config.max_length, mode='train')
dev_dataset = TextDataset(tokenizer, max_length=config.max_length, mode='dev')
test_dataset = TextDataset(tokenizer, max_length=config.max_length, mode='test')

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=config.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)

## Training, Evaluation and Metrics

In [3]:
from tqdm import tqdm
import json

def train(train_loader, val_loader, model, config):

    optimizer = AdamW(model.parameters(), lr=config.lr)
    
    model_name = '{}_B-{}_E-{}_Lr-{}_add'.format(config.model, config.batch_size,
                                                        config.epochs, config.lr)
    max_score = 0
    
    for epoch in range(config.epochs):
        model.train()

        preds_all = np.zeros((0, config.num_labels), dtype=int)
        labels_all = np.zeros((0, config.num_labels), dtype=int)
        loss_all, length = 0, 0

        for batch in tqdm(train_loader, desc='Training', colour='MAGENTA'):
            labels = batch['labels'].int().numpy()

            optimizer.zero_grad()
            for key, val in batch.items():
                batch[key] = val.to(config.device)
            outputs = model(**batch)

            loss, logits = outputs.loss, outputs.logits
            preds = get_preds(logits)
            preds_all = np.concatenate((preds_all, preds))
            labels_all = np.concatenate((labels_all, labels))
            loss_all += loss.item()
            length += 1

            loss.backward()
            optimizer.step()
            
        train_score = get_scores(preds_all, labels_all, loss_all, length, 'train')
        dev_score, _ =  evaluate(val_loader, model, config)
        f = open('{}/{}.all_scores.txt'.format(config.result_path, model_name), 'a')
        f.write(' ==================================================  Epoch: {}  ==================================================\n'.format(epoch))
        f.write('TrainScore: \n{}\nEvalScore: \n{}\n'.format(
                json.dumps(train_score), json.dumps(dev_score)))
        max_score = save_best(config, epoch, model_name,
                                  model, dev_score, max_score)
        print('End for {} epoch'.format(epoch))
        
def save_best(config, epoch, model_name, model, score, max_score):
    score_key = config.score_key
    curr_score = score[score_key]
    print('The epoch_{} {}: {}\nCurrent max {}: {}'.format(
        epoch, score_key, curr_score, score_key, max_score))

    if curr_score > max_score or epoch == 0:
        torch.save({
            'epoch': config.epochs,
            'model_state_dict': model.state_dict(),
        }, '{}/{}-BEST.tar'.format(config.checkpoint_path, model_name))
        return curr_score
    else:
        return max_score

def get_preds(logit):
    return (logit > 0).int().cpu().numpy()

def get_scores(preds, labels, loss_all=None, length=None, data_name='train'):
    score_dict = dict()
    union = preds | labels
    inter = preds & labels
    tp = np.sum(inter, axis=1)
    accuracy =  tp / np.sum(union, axis=1)
    precision = tp / (np.sum(preds, axis=1) + 1e-10) # 防止除0
    recall = tp / np.sum(labels, axis=1)
    f1_score = 2*(precision*recall)/(precision + recall + 1e-10)
    
    score_dict['accuracy'] = np.mean(accuracy)
    score_dict['precision'] = np.mean(precision)
    score_dict['recall'] = np.mean(recall)
    score_dict['f1_score'] = np.mean(f1_score)
    if loss_all is not None:
        score_dict['all_loss'] = loss_all / length

    print("Evaling on \"{}\" data".format(data_name))
    for s_name, s_val in score_dict.items():
        print("{}: {}".format(s_name, s_val))
    return score_dict

def evaluate(val_loader, model, config):
    model.eval()
    preds_all = np.zeros((0, config.num_labels), dtype=int)
    labels_all = np.zeros((0, config.num_labels), dtype=int)
    loss_all, length = 0, 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Evaling', colour='CYAN'):
            labels = batch['labels'].int().numpy()
            for key, val in batch.items():
                batch[key] = val.to(config.device)
            outputs = model(**batch)
            loss, logits = outputs.loss, outputs.logits
            preds = get_preds(logits)

            preds_all = np.concatenate((preds_all, preds))
            labels_all = np.concatenate((labels_all, labels))
            loss_all += loss.item()
            length += 1
    
    dev_score = get_scores(preds_all, labels_all, loss_all, length, 'dev')
    return dev_score, preds_all

## training process of bert

In [4]:
config.model = 'bert-base-uncased'
model = BertForSequenceClassification.from_pretrained(config.model, num_labels=config.num_labels).to(config.device)
train(train_loader, test_loader, model, config)

Training: 100%|██████████| 1357/1357 [01:17<00:00, 17.51it/s]


Evaling on "train" data
accuracy: 0.16295629166109565
precision: 0.17390011095568528
recall: 0.16738462719803426
f1_score: 0.166980639720203
all_loss: 0.15064760671438499


Evaling: 100%|██████████| 170/170 [00:03<00:00, 43.31it/s]


Evaling on "dev" data
accuracy: 0.34483446962717273
precision: 0.3729500644550949
recall: 0.34520299735888454
f1_score: 0.3541305816241977
all_loss: 0.10174957268378314
The epoch_0 f1_score: 0.3541305816241977
Current max f1_score: 0
End for 0 epoch


Training: 100%|██████████| 1357/1357 [01:19<00:00, 17.01it/s]


...

Evaling on "dev" data
accuracy: 0.5234230084147166
precision: 0.5675941280710808
recall: 0.5598089797923961
f1_score: 0.5506295681536136
all_loss: 0.1200662418761674
The epoch_8 f1_score: 0.5506295681536136
Current max f1_score: 0.5607333701304005
End for 8 epoch


Training: 100%|██████████| 1357/1357 [01:23<00:00, 16.20it/s]


Evaling on "train" data
accuracy: 0.900697611917377
precision: 0.9470840051311054
recall: 0.9148782922521693
f1_score: 0.9210825900522344
all_loss: 0.022604327550735777


Evaling: 100%|██████████| 170/170 [00:04<00:00, 42.00it/s]

Evaling on "dev" data
accuracy: 0.5161507278422701
precision: 0.5591947668535475
recall: 0.5561697684417418
f1_score: 0.5442601805244636
all_loss: 0.12801834975971896
The epoch_9 f1_score: 0.5442601805244636
Current max f1_score: 0.5607333701304005
End for 9 epoch


In [9]:
config.model = 'bert-large-uncased'
model = BertForSequenceClassification.from_pretrained(config.model, num_labels=config.num_labels).to(config.device)
train(train_loader, test_loader, model, config)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

Evaling on "train" data
accuracy: 0.25424053763604365
precision: 0.27473760199196123
recall: 0.2596847884512017
f1_score: 0.26174031054518354
all_loss: 0.1293997049891694


Evaling: 100%|██████████| 170/170 [00:10<00:00, 15.99it/s]


Evaling on "dev" data
accuracy: 0.4422486333763282
precision: 0.4785332595801732
recall: 0.4435999017259383
f1_score: 0.4545543885056815
all_loss: 0.09069331498707042
The epoch_0 f1_score: 0.4545543885056815
Current max f1_score: 0
End for 0 epoch


Training: 100%|██████████| 1357/1357 [04:09<00:00,  5.43it/s]


...

Evaling on "dev" data
accuracy: 0.5281739450893679
precision: 0.5698882132021142
recall: 0.5880474172348136
f1_score: 0.5624101713094561
all_loss: 0.13633491887765772
The epoch_8 f1_score: 0.5624101713094561
Current max f1_score: 0.5710337202306451
End for 8 epoch


Training: 100%|██████████| 1357/1357 [04:19<00:00,  5.22it/s]


Evaling on "train" data
accuracy: 0.9400341703140597
precision: 0.9747761651540955
recall: 0.951535744452123
f1_score: 0.9556017740572809
all_loss: 0.014264276805906861


Evaling: 100%|██████████| 170/170 [00:10<00:00, 15.60it/s]

Evaling on "dev" data
accuracy: 0.5152939008660402
precision: 0.556307966289559
recall: 0.5706037712671213
f1_score: 0.5476611650196318
all_loss: 0.1433068057631745
The epoch_9 f1_score: 0.5476611650196318
Current max f1_score: 0.5710337202306451
End for 9 epoch


## evaluation models

#### BERT

In [4]:
def test_data(model_path, test_loader, config):
    model = BertForSequenceClassification.from_pretrained(config.model, num_labels=config.num_labels).to(config.device)
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    print('metrics of {} after {} epochs of fine_tune'.format(config.model, checkpoint['epoch']))
    test_score, preds = evaluate(test_loader, model, config)
    return test_score, preds

config.model = 'bert-base-uncased'
model_path = "saved_dict/bert-base-uncased_B-32_E-10_Lr-2e-05_add-BEST.tar"
test_score, preds = test_data(model_path, test_loader, config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

metrics of bert-base-uncased after 10 epochs of fine_tune


Evaling: 100%|██████████| 170/170 [00:03<00:00, 43.69it/s]

Evaling on "dev" data
accuracy: 0.533843130028868
precision: 0.5794023708067003
recall: 0.5684693814876236
f1_score: 0.5607333701304005
all_loss: 0.11245935250292806


In [5]:
config.model = 'bert-large-uncased'
model_path = "saved_dict/bert-large-uncased_B-32_E-10_Lr-2e-05_add-BEST.tar"
test_score, preds = test_data(model_path, test_loader, config)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

metrics of bert-large-uncased after 10 epochs of fine_tune


Evaling: 100%|██████████| 170/170 [00:09<00:00, 18.37it/s]


Evaling on "dev" data
accuracy: 0.547616854001597
precision: 0.5932989373545784
recall: 0.5717093544622566
f1_score: 0.5710337202306451
all_loss: 0.1026441698346068


#### llama

In [6]:
def analyze_llm(result_path, name):
    llm_result = pd.read_csv(result_path)
    preds = np.zeros((0, config.num_labels), dtype=int)
    labels = np.zeros((0, config.num_labels), dtype=int)

    for pred, label in zip(llm_result['predicted_sentiment'], llm_result['sentiment_index']):
        pred = [json.loads(pred)]
        preds = np.append(preds, pred, axis=0)
        label = [convert_onehot(label.split(','))]
        labels = np.append(labels, label, axis=0)
    print('metrics of {}'.format(name))
    get_scores(preds, labels, data_name='test')

analyze_llm("result/goemotions_with_predictions_llama_3.1_8B_instruct.csv", 'llama_zero_shot')
print("-"*100)
analyze_llm("result/goemotions_with_predictions__llama_3.1_8B_instruct_sst_finetuned.csv", 'llama_fine_tuned')

metrics of llama_zero_shot
Evaling on "test" data
accuracy: 0.2096631042319268
precision: 0.2348565812690553
recall: 0.24006510656593577
f1_score: 0.2266416301566222
----------------------------------------------------------------------------------------------------
metrics of llama_fine_tuned
Evaling on "test" data
accuracy: 0.34401493414760415
precision: 0.36251765858661245
recall: 0.41290461273877527
f1_score: 0.36660261300023267
